In [1]:
import time
from selenium import webdriver
from random import randint, choice, uniform
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchFrameException, ElementClickInterceptedException

In [2]:
options = Options()
# chrome_options.add_argument("--headless")    ##uncoment to use a headless chrome (fatser performance when compared to with headless)
options.add_argument("--start-maximized")
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=options) ##uncomment if starting freshly or if path to chromedriver isn't knwon
driver = webdriver.Chrome(service=Service(r'C:\Users\Living Room\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe'),options=options)

In [3]:
def tap_and_select_random_option(base_locator,drop_locator_index,input_string=None):
    """ tap on the base_locator to input a string onto input_box 
        and select a random option from the dropdown list containing
        various options 
    input base_locator : locator to the input dropdownbox
    input drop_locator_index : the drop_down_list_index 
                                {2-> for the first dropbox}
    input input_string : string input which needs to be fed 
                        to the dropbox for loading the default values
    returns: None
    """
    try:
        drop_down_locator = '#vs' + str(drop_locator_index) + '__listbox'
        drop_list_locator = '#vs' + str(drop_locator_index) + '__combobox > div.vs__selected-options > input'
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, base_locator))).click()
        if input_string :
            driver.find_element(By.CSS_SELECTOR,drop_list_locator).clear()
            driver.find_element(By.CSS_SELECTOR,drop_list_locator).send_keys(input_string)
            time.sleep(2)
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, drop_down_locator)))
        drop_down_options = get_all_options_from_dropdown(drop_down_locator)
#         print(drop_down_options)                         ##uncomment for drop_down_options
        random_value = randint(0, len(drop_down_options[0])-1)           
        if input_string :
            drop_down_options[0] = drop_down_options[0][:-1]
            if len(drop_down_options[0]) <=2 :                         ## fail safe for the dropdown options having only 2 options
                random_value = randint(0, len(drop_down_options[0])-1)
            else : random_value = randint(0, len(drop_down_options[0])-2)  ## ensuring not selecting the half input text entered
        vs_random_locator = '#vs' + str(drop_locator_index) + '__option-' + str(random_value)
#         print(vs_random_locator)                         ##uncomment for random vs_combox_locator 
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, vs_random_locator))).click()
    except ElementClickInterceptedException:
        print(" cheking if chat_box is open for help ")
        if close_the_chat_box() : 
            print( " CLOSED SUCCESSFULLY")
            tap_and_select_random_option(base_locator,drop_locator_index,input_string=input_string)
        pass

In [4]:
def click_to_next_page(step=None):
    """Click's the next button 
        present at the bottom 
        of the page
        returns:None
    """
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > validationobserver > form > div.grid.grid-cols-2.gap-6.pt-12 > div.text-right > button')
#         driver.execute_script("arguments[0].click();", next_button)     ## could also be used inplace of below line
        next_button.click()
    except :
        if( step == 1):                     # just a fail safe check 
            driver.refresh()
            time.sleep(2)
            tap_and_select_random_option("#country",'1')
        print("try completing the required mandatory fields on the page!, refreshing page")

In [5]:
def get_current_step(try_once = 1):
    """load's the curent step 
        number on a given page
        input : max_tries untill it fails
        returns  : int value of current step
    """
    try :
        try_once -=1
        step_info = driver.find_element(By.CSS_SELECTOR , '#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div.-mx-4.sm\:-mx-12.border-b-2 > div > div.flex.justify-between.space-x-6.sm\:mr-12.mt-4.lg\:mt-0 > div.text-gray-700.hidden.sm\:block')
        print(f"Currently at step -> {step_info.text.split(' ')[1]}")
        return int(step_info.text.split(' ')[1])
    except NoSuchElementException :
        print("not able to retrieve the step info , retrying after a second")
        time.sleep(1)
        if try_once >= 0 : get_current_step(try_once)

In [6]:
def get_all_options_from_dropdown(locator):
    """ Returns all the possible options 
        under the given dropdown 
    input : ccs_selector locator for the 
            required dropdown
    output : list of options
    """
    drop_down_options = []
    drop_down_list = driver.find_element(By.CSS_SELECTOR, locator)
    drop_down_options.append(drop_down_list.text.split('\n'))
#     print(drop_down_list.text)                             ##uncomment to print all the drop_down options
    return drop_down_options

In [7]:
def go_back_a_step():
    """navigating back to the previous step
    inputs : None
    returns : None
    """
    step_back = driver.find_element(By.CSS_SELECTOR , '#__layout > div > main > div > div > div > validationobserver > form > div.grid.grid-cols-2.gap-6.pt-12 > div:nth-child(1) > button')
    step_back.click()

In [8]:
def close_the_chat_box():
    """ tries to minimise the chot_box window
    popped up on the left bottom and returns control
    back to the  main page
    inputs : None
    return : True is chatbox is closed
    """
    try:
        driver.switch_to.frame("siqiframe")
        # driver.switch_to.default_content()
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'body > div.win_close.sqico-larrow'))).click()
        time.sleep(2)
        driver.switch_to.default_content()
        print("FOUND THE CHAT BOX INTERCEPTING THE MAIN PAGE")
        return True
    except (NoSuchFrameException,NoSuchElementException,ElementClickInterceptedException):
        driver.switch_to.default_content()
        print("No such frame , or chat box is colsed already!")
        pass

In [9]:
def update_the_score(locator, score):
    """ waits for the locator input box to be clickable
        and clear's any previous data entered , enters the data-> score
        input : locator to the input box
        input : score to be entered into input box
        return : None"""
    try:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, locator))).send_keys(Keys.CONTROL + "a")
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, locator))).send_keys(Keys.DELETE)
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, locator))).send_keys(score)
    except :
        print("Locator issue!, check the locator asssigned")

In [10]:
def check_if_proper_page(desired_step):
    """checks if the current page is on given desired step 
    input : desired_step number 
    returns : true if current_step and desired step are same"""
    if get_current_step() == desired_step :
        print("Got into proper page for desired step")
        return True
    elif get_current_step() > desired_step: 
        go_back_a_step()
        print("Stepping a page back")
        check_if_proper_page(desired_step)
    else :
        print("Please complete the mandatory information over current page before navigating to next")
        return False

In [11]:
def first_page(max_tries=3):
    """ first page actions
    input : maximum_tries before it fails
    returns : true if successful
    """
    try:
        max_tries -= 1
        check_if_proper_page(1)
        tap_and_select_random_option("#country",'1')
        print("Selected random country")
        close_the_chat_box()
        tap_and_select_random_option("#area_of_study",'2','eng')
        print("Selected random aread_of_study")
        close_the_chat_box()
        click_to_next_page()
        print("navigated to next page")
        return True
    except TimeoutException :
        driver.get(base_page)   ## at first fail tries to reload the page
        if (max_tries >= 0):
            first_page(max_tries)
        else :
            print("Maximum tries reached for first page ")

In [12]:
def second_page(max_tries=2):
    """ second page actions
    input : maximum_tries before it fails
    returns : true if successful
    """
    try:
        max_tries -= 1
        if not check_if_proper_page(2):
            print("Filing out the first step first !")
            first_page()
        tap_and_select_random_option("#college",'4','tech')
        print("Selected random college")
        tap_and_select_random_option("#major",'6','Elec')
        print("Selected random major_of_study")
        driver.find_element(By.CSS_SELECTOR,'#marks').clear()
        random_cgpa = round(uniform(5,10), 2)
        update_the_score('#marks',random_cgpa)
        close_the_chat_box()
        click_to_next_page()
        print("clicked on next button ")
        return True
    except TimeoutException :
        driver.refresh()
        time.sleep(2)
        if (max_tries >= 0):
            second_page(max_tries)
        else :
            print("Maximum tries reached for second page ")

In [13]:
def fill_the_english_test_scores():
    """ fills the scores on the second page by randomly
        selecting the english tests
        input : None
        returns : None
        """
    english_test_name, english_locator_index = choice(list(exam_dict.items()))
    english_locator = ' #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pr-4 > div.grid.grid-cols-3.sm\:grid-cols-3.gap-2.justify-center.items-center.pt-2 > div:nth-child(' + str(english_locator_index) + ') > div' 
    print(english_test_name, english_locator)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, english_locator))).click()
    english_score = 0
    input_value_locator = '#'+ english_test_name + '_overall_score'
    print(input_value_locator)
    if( english_test_name == 'toefl') :
        english_score = randint(60,120)
    elif english_test_name == 'pte' :
        english_score = randint(45,90)
    elif english_test_name == 'ielts' :
        english_score = randint(5,9)
    else  :
        print( "select the proper exam section ! ")
    update_the_score(input_value_locator, english_score)

In [14]:
def fill_the_aptitude_test_scores():
    """ fills the scores for randomly selected aptitude 
        tests 
        inputs : None
        return : None 
    """
    aptitude_test_name, aptitude_locator_index = choice(list(aptitude_dict.items()))
    aptitude_locator = '#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pl-8 > div.pt-2 > div.grid.grid-cols-3.md\:grid-cols-3.gap-2.justify-center.items-center > div:nth-child(' + str(aptitude_locator_index) + ') > div'
    print(aptitude_test_name, aptitude_locator)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, aptitude_locator))).click()
    if( aptitude_test_name == 'gre') :
#         aptitude_score = [randint(85,170),randint(85,170),randint(3,6)]
        update_the_score('#verbal_score', randint(130,170))
        update_the_score('#quant_score', randint(130,170))
        update_the_score('#awa_score', randint(3,6))
    elif aptitude_test_name == 'gmat' :
        update_the_score('#total_gmat_score', randint(450,800))
    elif aptitude_test_name == 'none' :
        aptitude_score = None
    else  :
        print( "select the proper exam section ! ")

In [15]:
root_page = 'https://yocket.com'
base_page  = 'https://yocket.com/college-finder/masters'

In [16]:
exam_dict = {
    'ielts' : '2',
    'pte' : '3',
    'toefl' : '1'
    }

In [17]:
aptitude_dict = {
    'gre' : '1',
    'gmat' : '2',
    'none' : '3'
}

In [18]:
def third_page(max_tries=1):
    """ third page actions
    input : maximum_tries before it fails
    returns : true if successful
    """
    try:
        max_tries -= 1
        if not check_if_proper_page(3):
            print("Filing out the second step first !")
            second_page()
        fill_the_english_test_scores()
        print("Filled random scores for random english test")
        fill_the_aptitude_test_scores()
        print("Filled random aptitude scores")
        close_the_chat_box()
        click_to_next_page()
        print("clicked on next button ")
        return True
    except TimeoutException :
        driver.refresh()
        time.sleep(2)
        if (max_tries >= 0):
            third_page(max_tries)
        else :
            print("Maximum tries reached for third page ")

In [19]:
def fourth_page(max_tries=1):
    """ fourth page actions
    input : maximum_tries before it fails
    returns : true if successful
    """
    try:
        max_tries -= 1
        if not check_if_proper_page(4):
            print("Filing out the third step first !")
            third_page()
        update_the_score('#work_exp',randint(5,50))
        print("entered random scores for work experience")
        update_the_score('#project',randint(0,10))
        print("entered random project's done")
        close_the_chat_box()
        click_to_next_page()
        print("clicked on next button ")
        return True
    except TimeoutException :
        driver.refresh()
        time.sleep(2)
        if (max_tries >= 0):
            fourth_page(max_tries)
        else :
            print("Maximum tries reached for fourth page ")

In [20]:
relevance_dict = {
    'relevance' : '0',
    'cost' : '1'
}

In [21]:
def get_list_of_all_universities(locator):
    """get all the univeristies from the locator
    input : locator to the result 
    return : list of universities
    """
    output = []
    universities = driver.find_element(By.CSS_SELECTOR, locator)
#     print(universities.text)
    raw_university = universities.text.split('Shortlist')[1:]
    for r in raw_university:
        output.append(r.split('\n')[1:])
    return output

In [22]:
def check_if_results_zero(locator):
    """
    Check if we have zero results for given 
    combinations of inputs
    returns : True if no universities were matched
              Flase if we have matching universities
    """
    try : 
        result = driver.find_element(By.CSS_SELECTOR, locator)
        if result.text == 'Uh-oh! That spell didn’t work': return True
        else: return False
    except :
        print ('seomthing wrong with the locator')

In [27]:
def scrape_on_50_opacity():
    """scraping on the output window where the results are hidden 
         return : ambitiious universites if we have , else  target universities
         """
    try :
        if driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > p').text == "Ambitious Universities (0)" :
    #         error_message = driver.find_element(By.CSS_SELECTOR,'#__layout > div > main > div > div > div > div.text-center.lg\:text-left.text-gray-700 > div > p.text-xl.text-gray-900.text-opacity-75.font-semibold').text
            print("Our search couldn't help find a better match ! , here is the message ")
    #         print(error_message)
            print("Sorry, we couldn’t find universities under the ambitious category that match your profile."
                  "Try our Premium plans (spells) and get expert guidance! . checking for any Target Universities"
                  "\nHere's your other options! TARGET Universities")
        else :
            val  = Select(driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > div:nth-child(5) > div.flex.justify-between.items-center > div > select'))
            val.select_by_value(relevance_dict['cost'])
            ambitious_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(5) > div.grid.grid-cols-1.sm\:grid-cols-2.lg\:grid-cols-3.gap-4.sm\:gap-6')
            print(ambitious_universities)
            return ambitious_universities
        target_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(7) > div')
    #     safe_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(9) > div > div')
        safe_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(9) > div')

        print(target_universities)
        print("\nHere's your other options! Safe Universities")
        print(safe_universities)
        return target_universities
    except NoSuchElementException:
        print("webpage didn't load after a second,waiting for a second more")
        time.sleep(1)
        scrape_on_50_opacity()

In [24]:
def final_result(): 
    """ append all the results found"""
    if check_if_results_zero('#__layout > div > main > div > div > div > div > p.text-2xl.md\:text-3xl.text-gray-900.text-opacity-75.font-semibold'):
        print("FOUND ZERO RESULTS , PROBABLY YOU NEED TO RECONSIDER YOUR CARRER OPTION")
        pass
    else : 
        Ideal_match = scrape_on_50_opacity()
        print("\n============================================\nIDEAL UNIVERSITY FOUND IS: {} \nLOCATED in {}\nMAJOR in {}\nFEE {}".format (Ideal_match[0][0],Ideal_match[0][1],Ideal_match[0][2],Ideal_match[0][3]))

In [25]:
def get_to_final_page():
    """ stiching all together
    """
    driver.get(base_page)
    first_page()
    second_page()
    third_page()
    fourth_page()

In [28]:
get_to_final_page()
time.sleep(1)
final_result()

Currently at step -> 1
Got into proper page for desired step
Selected random country
No such frame , or chat box is colsed already!
Selected random aread_of_study
No such frame , or chat box is colsed already!
navigated to next page
Currently at step -> 2
Got into proper page for desired step
Selected random college
Selected random major_of_study
No such frame , or chat box is colsed already!
clicked on next button 
Currently at step -> 3
Got into proper page for desired step
pte  #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pr-4 > div.grid.grid-cols-3.sm\:grid-cols-3.gap-2.justify-center.items-center.pt-2 > div:nth-child(3) > div
#pte_overall_score
Filled random scores for random english test
gmat #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pl-8 > div.pt-2 > div.grid.grid-cols-3.

### experimental 

In [436]:
def scroll_to_top_of_page():
    top = driver.find_element(By.CSS_SELECTOR, '#__layout > div > div.w-full > div > main > div > div > div > div.pb-5.text-center.lg\:text-left > div')
    driver.execute_script("return arguments[0].scrollIntoView(true);", top)

In [438]:
scroll_to_top_of_page()

In [283]:
def sign_up_for_yocket():
    """ sign_up on the portal
    """
    driver.get('https://yocket.com/login')

In [31]:
driver.find_element(By.CSS_SELECTOR,'#__layout > div > main > div > div > div > p').text

'Ambitious Universities (6)'

In [34]:
scrape_on_50_opacity()

[['Brunel University', 'United Kingdom', 'MSc Advanced Electronic and Electrical Engineering', '£20,450/year', '12 months', 'See all admits', 'Get Guidance', ''], ['University of Sheffield', 'United Kingdom', 'MSc(Eng) Advanced Electrical Machines, Power Electronics and Drives', '£24,950/year', '12 months', 'See all admits', 'Get Guidance', ''], ['University of Birmingham', 'United Kingdom', 'MSc Electrical Power Systems', '£25,740/year', '12 months', 'See all admits', 'Get Guidance']]


[['Brunel University',
  'United Kingdom',
  'MSc Advanced Electronic and Electrical Engineering',
  '£20,450/year',
  '12 months',
  'See all admits',
  'Get Guidance',
  ''],
 ['University of Sheffield',
  'United Kingdom',
  'MSc(Eng) Advanced Electrical Machines, Power Electronics and Drives',
  '£24,950/year',
  '12 months',
  'See all admits',
  'Get Guidance',
  ''],
 ['University of Birmingham',
  'United Kingdom',
  'MSc Electrical Power Systems',
  '£25,740/year',
  '12 months',
  'See all admits',
  'Get Guidance']]

In [35]:
final_result()

seomthing wrong with the locator
[['Brunel University', 'United Kingdom', 'MSc Advanced Electronic and Electrical Engineering', '£20,450/year', '12 months', 'See all admits', 'Get Guidance', ''], ['University of Sheffield', 'United Kingdom', 'MSc(Eng) Advanced Electrical Machines, Power Electronics and Drives', '£24,950/year', '12 months', 'See all admits', 'Get Guidance', ''], ['University of Birmingham', 'United Kingdom', 'MSc Electrical Power Systems', '£25,740/year', '12 months', 'See all admits', 'Get Guidance']]
 IDEAL UNIVERSITY FOUND IS United Kingdom
